In [ ]:
import jsonref,json
from fastcore.utils import *
from fastcore.foundation import *
from fastcore.meta import *

import pprint,inspect
from inspect import signature,Parameter

In [ ]:
js = jsonref.load(open('../openapi/api.github.com.json'))
# js['paths']['/repos/{owner}/{repo}/git/ref/{ref}']

In [ ]:
_DOC_URL = 'https://docs.github.com/'

In [ ]:
def _api_js():
    url = 'https://github.com/github/rest-api-description/blob/main/descriptions/api.github.com/api.github.com.json?raw=true'
    return urljson(url)

In [ ]:
def build_funcs(nm='funcs.py'):
    def _get_detls(o):
        data = nested_idx(o, *'requestBody content application/json schema properties'.split()) or {}
        url = o['externalDocs']['url'][len(_DOC_URL):]
        return (o['operationId'], o['summary'], url, list(data.keys()))
    
    js = _api_js()
    pre = '/repos/{owner}/{repo}'
    paths = {o[len(pre):]:v for o,v in js['paths'].items() if o.startswith(pre)}
    _funcs = [(path, verb) + _get_detls(detls)
              for path,verbs in paths.items() for verb,detls in verbs.items()]
    Path(nm).write_text("funcs = " + pprint.pformat(_funcs, width=360))

In [ ]:
# paths,comps = itemgetter('paths','components')(_api_js())
# params = comps['parameters']
# build_funcs()

In [ ]:
from funcs import funcs

In [ ]:
funcs[0]

('', 'get', 'repos/get', 'Get a repository', 'v3/repos/#get-a-repository', [])

In [ ]:
#export
GH_HOST = "https://api.github.com"

In [ ]:
import string
_fmt = string.Formatter()

def stringfmt_names(s:str)->L:
    "Unique brace-delimited names in `s`"
    return L(_fmt.parse(s)).itemgot(1).unique().filter()

In [ ]:
stringfmt_names('/pulls/{pull_number}/reviews/{review_id}')

(#2) ['pull_number','review_id']

In [ ]:
def _call(self, *args, **kwargs):
    # TODO: post data
    for a,b in zip(args,self.params): kwargs[b]=a
    kw = {p:kwargs[p] for p in self.params}
    path = self.path.format(**kw)
    return self.client(path)

_self_param = [Parameter('self', Parameter.POSITIONAL_OR_KEYWORD)]

def _replace_sig(f, path_args, data_args):
    p = [Parameter(o,               kind=Parameter.POSITIONAL_OR_KEYWORD) for o in path_args]
    d = [Parameter(o, default=None, kind=Parameter.POSITIONAL_OR_KEYWORD) for o in data_args]
    f.__signature__ = signature(f).replace(parameters=_self_param+p+d)
    return f

In [ ]:
class GhVerb:
    __slots__ = 'path,verb,tag,name,summary,url,params,data,client,__call__,__doc__'.split(',')
    
    def __init__(self, path, verb, oper, summary, url, data, client):
        tag,name = oper.split('/')
        name = name.replace('-','_')
        params = filter_ex(stringfmt_names(path), in_(('owner','repo')), True)
        if params and not params[0]: breakpoint()
        f = _replace_sig(copy_func(_call), params, data)
        __doc__ = summary
        __call__ = MethodType(f, self)
        store_attr(self.__slots__)
    
    def _repr_markdown_(self):
        params = ','.join(self.params+self.data)
        return f"[{self.tag}/{self.name}]({_DOC_URL}{self.url})({params}): {self.summary}"
    __repr__ = _repr_markdown_

In [ ]:
class _GhVerbGroup(AttrDict):
    def __init__(self, verbs): super().__init__(**{o.name:o for o in verbs})
    def _repr_markdown_(self): return "\n".join(f'- [{k}]({_DOC_URL}{v.url})' for k,v in self.items())

In [ ]:
#export
class GhApi:
    def __init__(self, owner, repo, token):
        funcs_ = L(funcs).starmap(GhVerb, client=self)
        self._fs = {k:_GhVerbGroup(v) for k,v in groupby(funcs_, 'tag').items()}
        self._headers = { 'Authorization' : 'token ' + token,
                         'Accept': 'application/vnd.github.v3+json'}
        self.owner,self.repo = owner,repo
        self.repo_url = f"{GH_HOST}/repos/{owner}/{repo}"

    def __dir__(self): return super().__dir__() + list(self._fs)

    def __getattr__(self,k):
        if k in self._fs: return self._fs[k]
        raise AttributeError(k)

    def __call__(self, path, **data):
        "Call GitHub API `path`"
        path = f"{self.repo_url}{path}"
        return dict2obj(do_request(path, headers=self._headers, **data))

In [ ]:
api = GhApi('fastai', 'fastcore', os.environ['FASTRELEASE_TOKEN'])

In [ ]:
api.git.get_ref('heads/master')

- ref: refs/heads/master
- node_id: MDM6UmVmMjI1NDYwNTk5OnJlZnMvaGVhZHMvbWFzdGVy
- url: https://api.github.com/repos/fastai/fastcore/git/refs/heads/master
- object: 
  - sha: 569a67ed66fae0ed12015042fcfe25cd51671082
  - type: commit
  - url: https://api.github.com/repos/fastai/fastcore/git/commits/569a67ed66fae0ed12015042fcfe25cd51671082

In [ ]:
api.repos.create_webhook

[repos/create_webhook](https://docs.github.com/rest/reference/repos#create-a-repository-webhook)(name,config,events,active): Create a repository webhook

In [ ]:
api.git

- [create_blob](https://docs.github.com/rest/reference/git#create-a-blob)
- [get_blob](https://docs.github.com/rest/reference/git#get-a-blob)
- [create_commit](https://docs.github.com/rest/reference/git#create-a-commit)
- [get_commit](https://docs.github.com/rest/reference/git#get-a-commit)
- [list_matching_refs](https://docs.github.com/rest/reference/git#list-matching-references)
- [get_ref](https://docs.github.com/rest/reference/git#get-a-reference)
- [create_ref](https://docs.github.com/rest/reference/git#create-a-reference)
- [update_ref](https://docs.github.com/rest/reference/git#update-a-reference)
- [delete_ref](https://docs.github.com/rest/reference/git#delete-a-reference)
- [create_tag](https://docs.github.com/rest/reference/git#create-a-tag-object)
- [get_tag](https://docs.github.com/rest/reference/git#get-a-tag)
- [create_tree](https://docs.github.com/rest/reference/git#create-a-tree)
- [get_tree](https://docs.github.com/rest/reference/git#get-a-tree)

In [ ]:
api.git.get_ref.path

'/git/ref/{ref}'

In [ ]:
api.issues.list_for_repo()[0]

- url: https://api.github.com/repos/fastai/fastcore/issues/217
- repository_url: https://api.github.com/repos/fastai/fastcore
- labels_url: https://api.github.com/repos/fastai/fastcore/issues/217/labels{/name}
- comments_url: https://api.github.com/repos/fastai/fastcore/issues/217/comments
- events_url: https://api.github.com/repos/fastai/fastcore/issues/217/events
- html_url: https://github.com/fastai/fastcore/issues/217
- id: 747092503
- node_id: MDU6SXNzdWU3NDcwOTI1MDM=
- number: 217
- title: delegates doesn't pull the __init__ of a class
- user: 
  - login: muellerzr
  - id: 7831895
  - node_id: MDQ6VXNlcjc4MzE4OTU=
  - avatar_url: https://avatars0.githubusercontent.com/u/7831895?v=4
  - gravatar_id: 
  - url: https://api.github.com/users/muellerzr
  - html_url: https://github.com/muellerzr
  - followers_url: https://api.github.com/users/muellerzr/followers
  - following_url: https://api.github.com/users/muellerzr/following{/other_user}
  - gists_url: https://api.github.com/users/muellerzr/gists{/gist_id}
  - starred_url: https://api.github.com/users/muellerzr/starred{/owner}{/repo}
  - subscriptions_url: https://api.github.com/users/muellerzr/subscriptions
  - organizations_url: https://api.github.com/users/muellerzr/orgs
  - repos_url: https://api.github.com/users/muellerzr/repos
  - events_url: https://api.github.com/users/muellerzr/events{/privacy}
  - received_events_url: https://api.github.com/users/muellerzr/received_events
  - type: User
  - site_admin: False
- labels: 

- state: open
- locked: False
- assignee: None
- assignees: 

- milestone: None
- comments: 0
- created_at: 2020-11-20T02:43:36Z
- updated_at: 2020-11-20T02:44:23Z
- closed_at: None
- author_association: NONE
- active_lock_reason: None
- body: There is currently a bug with tab completion when it comes to classes and delegates. A prime example is `Datasets`. It's setup as:
```python
@delegates(TfmdLists)
class Datasets(FilteredBase)
```
When performing a tab-hint to see the possible parameters, we should see both its parameters as well as `TfmdLists` for the kwargs. Instead we currently see: 
![image](https://user-images.githubusercontent.com/7831895/99751341-0582c900-2ab0-11eb-914e-304b8ef23278.png)

Another example is here is the results of `doc`:
![image](https://user-images.githubusercontent.com/7831895/99751388-18959900-2ab0-11eb-98a3-87b4e3daa3ca.png)


You can see the `kwargs` were present in the first, dropped in the last but only a *part* of `TfmdLists`' parameters were added. For extra clarity I will manually adjust the `delegates()` to include `TfmdLists`' `__init__`, where we can see it working as expected:

![image](https://user-images.githubusercontent.com/7831895/99751484-4549b080-2ab0-11eb-8d9f-608c93c6abb9.png)

- performed_via_github_app: None

In [ ]:
def _tag_date(self, tag):
    try: tag_d = self.gh(f"git/ref/tags/{tag}")
    except HTTPError: raise Exception(f"Failed to find tag {tag}")
    commit_d = self.gh(tag_d.object.url, complete=True)
    self.commit_date = commit_d.committer.date
    return self.commit_date

In [ ]:
def release(self):
    "Tag and create a release in GitHub for the current version"
    run(f'git tag {ver}')
    run('git push --tags')
    run('git pull --tags')
    self.gh("releases", post=True, tag_name=ver, name=ver, body=notes)

## Export -